![for sale image, from https://time.com/5835778/selling-home-coronavirus/](https://api.time.com/wp-content/uploads/2020/05/selling-home-coronavirus.jpg?w=800&quality=85)

# Building a Predictive Model for Housing Sales
## Using Linear Regression for Mom & Pop Realty

## Overview and Business Problem

For the second phase of Flatiron's Live Data Science program we were tasked with developing a multiple linear regression model. This model would predict the price of houses in King County, Washington using data from the King County reality dataset. We decided to develop this model for a small realty business named "Mom and Pop Reality". The goal is to provide an accurate prediction for the price of their client's home before puttting it on the market. Client's will always want to get the most money for their home possible. However, realty firms will quickly find themselves with a poor reputation and out of business if a they are misleading or dishonest in their assessed target price. Assuming the firm is acting on good faith and want to provide an accurate assessment, their prediction model must be flexible to the market to continue being competitive in the market place. <br><br>
With these concerns in mind, we set out to explore the features in the data set to design our model, explore correlations between different features and the sale price of the home, and use the features with strong correlations to develop a model to achieve our goal. We made sure to normalize our data using a log transformation and scale our data for consistent analysis. As we concluded our analysis, we discovered the most important features to predicting sale price was the size of the home, the size of the lot, the number of bedrooms, and the condition and grade of the home. We recommend Mom & Pop Realty take these features into consideration when assessing the values of client's homes. 

## Data Understanding

This dataset contains house sale prices for King County,Washington. It includes homes sold between May 2014 and May 2015. The data was gathered from King County GIS Open Data. The data represents different features of homes in King County. The data is widely varied, as is to be expected. The data states when the house was built and if it was renovated as well as the date of sale. The data includes counts on floors, bathrooms and bedrooms. Also included in grade and condition of the home. The data also includes waterfront property designation and data on view on different landmarks from the property. The data inlcudes information on basement, living and lot area. The data also includes information regarding living and lot area of the closest 15 properties. Additionally, there is also locational data including zip code and latitude and longitudinal of the property. 

Given these features, the target variable will be the sale price of the home, as the goal is to build a model to predict price. We selected features related to the house as the important predictor features. These include continuous data such as the home and lot size, ordinal data such as the number of bedrooms and bathrooms, and categorical data such as the grade and condition of the homes.

In [ ]:
# import relevant libraries
import numpy as np
import pandas as pd
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm
from statsmodels.formula.api import ols

import warnings
warnings.filterwarnings('ignore')

## Exploring the data

In [ ]:
#what data types are in the data?
data = pd.read_csv('data/kc_house_data.csv')
data.info()

In [ ]:
# looking for any null values in the data.
data.isna().sum()

In [ ]:
#dropping duplicates from the data set based on their id value.
data.drop_duplicates(subset='id', inplace=True)

In [ ]:
# 'sqft_basement' is an object type?
# exploring the area of basements
data['sqft_basement'].value_counts()

In [ ]:
# '?' is a strange value, we'll make that 0
data.loc[data['sqft_basement'] == '?','sqft_basement'] = 0.0

In [ ]:
# orginally this data was stored as text (object), 
# we transform the data type to float to use later for anaylsis
# check to ensure the "?" is now 0.0
data['sqft_basement'] = data['sqft_basement'].map(float)

In [ ]:
#exploring the distribution of the features
data.hist(figsize=(20,20));

Some of these features look like they could use some log processing. Running a log transformation will help normalize the data for better regression analysis down the line.

In [ ]:
#running the log transformation. creates a new data frame.
datalog = pd.DataFrame()
log_cols = ['id','price','sqft_living','sqft_lot','sqft_above', 'sqft_basement']
for col in log_cols:
    if col == 'id':
        datalog[col] = data[col]
        continue
    datalog[f'{col}_log'] = data[col].map(lambda x: np.log(x))

In [ ]:
#merges the orginal and new dataframe
data = pd.merge(data,datalog,on='id')

In [ ]:
#checking the columns to ensure all data is collected.
data.columns

## Choice of included features

Since there were so many available predictors, we decided to take a subset of them. We based this decision on a desire to make the model predictive only on the features of the house itself, and not so much the location information. Furthermore, we did some initial exploration with location information, but it didn't really go anywhere so we scrapped the idea. See the notebook in 'notebooks/Dave/King_County_Map.ipynb'.

Firstly, because the price is so skewed, we wanted to use log(price) as our output variable, since it will be more normally distributed and thus will follow the assumptions of linear regression a bit better (residuals will be more normal as well). For downstream understanding purposes, we kept the original 'price' as well.

Secondly, we looked at 'grade', 'condition', 'view', 'sqft_living_log', 'sqft_lot_log', 'sqft_above_log', 'sqft_basement_log', 'bedrooms', 'bathrooms', and 'floors' as potentially important predictors of price.


In [ ]:
#reducing our features to consider for analyzation
rel_cols = ['price','price_log', 'grade', 'condition', 'view', 'sqft_living_log',
            'sqft_lot_log', 'sqft_above_log', 'sqft_basement_log', 
            'bedrooms', 'bathrooms', 'floors','sqft_living',
            'sqft_lot', 'sqft_above']
data = data[rel_cols]

## Data cleaning

In this section we will transform and remove some feature to make the data easier to analyze.

In [ ]:
#looking at some statistical information of the numerical dataframe columms
data.describe()

In [ ]:
# looking at the top 15 properties according to each feature
for col in data.columns:
    print(f'\n{col}:\n')
    print(data.sort_values(by=col,ascending=False).head(15))

33 bedrooms is pretty crazy and not highly correlated with a high price. Based on the data, we are unsure if this data is a typo or an honest outlier. We decided to drop this property.

In [ ]:
data = data[data['bedrooms'] != 33]

In [ ]:
# looking at the value counts for view we see none composes 89.8% of the column.
data['view'].value_counts()

In [ ]:
data['view'].value_counts()[0]/len(data)

In [ ]:
# delete 'view' column since not much information given
data.drop(columns='view', inplace=True)

In [ ]:
#here we have the new distribution of home prices in the data set
fig, ax = plt.subplots(figsize= (12,8))
ax = sns.histplot(data["price"]/1000)

ax.set_title("Home prices in King County",fontsize=40)
ax.set_xlabel("Price (in thousands)",fontsize=30)
ax.set_ylabel("Number of homes",fontsize=30)
ax.tick_params(labelsize=20)
ax.xaxis.set_major_formatter('${x:1.0f}')
plt.savefig("House_prices.png")

In [ ]:
# median house sale price
data['price'].median()

## Transforming categorical data
#### 'Grade' to 4 categories: Low, Average, Above Average, and Excellent

In [ ]:
# Lows including 3 Poor, 5 Fair and 6 Low Average
data['grade'].replace('3 Poor','Low Grade', inplace=True)
data['grade'].replace('5 Fair','Low Grade', inplace=True)
data['grade'].replace('4 Low','Low Grade', inplace=True)
data['grade'].replace('6 Low Average','Low Grade', inplace=True)

# Average including 7 Average 
data['grade'].replace('7 Average','Average Grade', inplace=True)

# Average Above including 8 Good,9 Better
data['grade'].replace('8 Good','Above Average Grade', inplace=True)
data['grade'].replace('9 Better','Above Average Grade', inplace=True)

# Excellent including 10 Very Good, 11 Excellent,12 Luxury and 13 Mansion
data['grade'].replace('10 Very Good','Excellent Grade', inplace=True)
data['grade'].replace('11 Excellent','Excellent Grade', inplace=True)
data['grade'].replace('12 Luxury','Excellent Grade', inplace=True)
data['grade'].replace('13 Mansion','Excellent Grade', inplace=True)

In [ ]:
#feature breakdown
data['grade'].value_counts(normalize=True)

#### 'Condition' to only 2 categories: Low-Average, and Good

In [ ]:
#looking at feature breakdown to decide how to transform the data
data['condition'].value_counts(normalize=True)

In [ ]:
# Low-Average including Poor, Fair, Average
data['condition'].replace('Poor','Low-Average Condition', inplace=True)
data['condition'].replace('Fair','Low-Average Condition', inplace=True)
data['condition'].replace('Average','Low-Average Condition', inplace=True)

# Good including Good and Very Good
data['condition'].replace('Good','Good Condition', inplace=True)
data['condition'].replace('Very Good','Good Condition', inplace=True)

#data is better distribued when grouped
data['condition'].value_counts(normalize=True)

In [ ]:
#exploring the means and counts for grades per condition
data.groupby(by=['condition','grade']).agg(['mean','count'])['price_log']

In [ ]:
#exploring the means and counts for grades per condition
data.groupby(by=['grade','condition']).agg(['mean','count'])['price']

In [ ]:
fig,ax = plt.subplots(figsize = (25,10))
sns.violinplot(x='grade',y=(data['price']/1000), hue= "condition", data = data, palette="rocket", 
               order=["Low Grade", "Average Grade", "Above Average Grade", "Excellent Grade"], 
               hue_order = ["Low-Average Condition", "Good Condition"])

ax.set_title("Grade and Condition of Homes",fontsize=50)
ax.set_xlabel("Grade",fontsize=40)
ax.set_ylabel("Price (in Thousands)",fontsize=40)
ax.tick_params(labelsize=30)
ax.yaxis.set_major_formatter('${x:1.0f}')
ax.legend(loc="upper left", borderaxespad=0., title = "Condition",fontsize=20, title_fontsize=30)
plt.savefig("Grade_Condition.png")

## Observations of Grad and conditions of Homes
Home price increases as grade improves <br>
Home price increases as condition improves <br>
Home price increases as condition improves within grade

### One-Hot Encoding the categorical data
He break out our categorical featsures (Condition and Grade) into individaul columns using SkLearn's Onehotencoding function and drop the first column to avoid multicollinearity issues

In [ ]:
cond = data[["condition"]]
ohe = OneHotEncoder(categories="auto", handle_unknown="error", sparse=False)
ohe.fit(cond)
cond_encod = ohe.transform(cond)
cond_encod = pd.DataFrame(
    # Pass in NumPy array
    cond_encod,
    # Set the column names to the categories found by OHE
    columns=ohe.categories_[0],
    # Set the index to match X_train's index
    index= data.index
)
cond_encod.drop(columns='Low-Average Condition', inplace=True)

In [ ]:
grade = data[["grade"]]
ohe = OneHotEncoder(categories="auto",handle_unknown="ignore", sparse=False)
ohe.fit(grade)
grade_encod = ohe.transform(grade)
grade_encod = pd.DataFrame(
    # Pass in NumPy array
    grade_encod,
    # Set the column names to the categories found by OHE
    columns=ohe.categories_[0],
    # Set the index to match X_train's index
    index= data.index
)
grade_encod.drop(columns='Average Grade', inplace=True)

In [ ]:
#drops the orginal columns because the data wee need is now split into columns
data = pd.concat([data, cond_encod, grade_encod], axis=1)\
        .drop(columns=['condition','grade'])

### Changing sqft_basement to a binomial variable "has_basement"

In [ ]:
# replace sqft_basement with has_basement: True (1) / False (0)
data['sqft_basement_log'] = data['sqft_basement_log'].map(lambda x: 1 if x > 0 else 0)
data.rename(columns={'sqft_basement_log':'has_basement'}, inplace=True)

In [ ]:
#pair plot visually exploring the relationship between the features
# sns.pairplot(data)

In [ ]:
#exploring the distributions of our data
fig, axes = plt.subplots(8,3, figsize=(30,30))
for i, col in enumerate(data.columns):
    sns.histplot(data=data, x=col, kde=True, ax=axes[i//3,i%3]);

In [ ]:
#exploring the correlations between the price and features
data.corr()

In [ ]:
data.columns

In [ ]:
#an organized series showing the correlations to price
datacorrp = data[['price_log', 'has_basement', 'bedrooms', 'bathrooms', 'floors',
       'sqft_living', 'sqft_lot', 'sqft_above',
       'Good Condition', 'Above Average Grade', 'Excellent Grade',
       'Low Grade']]
datacorr = datacorrp.corr().sort_values('price_log',ascending=False)['price_log']
datacorr = datacorr.drop(index=['price_log'])
datacorr

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
sns.barplot(x=datacorr.values, y=datacorr.index, orient='h', color='b')
ylabels = [item.capitalize() for item in datacorr.index]
ylabels = [item.split('_') for item in ylabels]
ylabels = [' '.join(item) for item in ylabels]
ax.set_yticklabels(ylabels, size=16)
ax.tick_params(axis='x', which='major', labelsize=18)
ax.set_xlabel('Correlation to Sale Price', size=18)
ax.set_ylabel('House Feature', size=18)
ax.set_title('House Feature Correlations', size=20);

In [ ]:
#exploring the correlations between features to look for multicollinearity issues
datafeat = data.drop(columns=['price','price_log'])
datafeat.corr()

In [ ]:
# analyzing the high correlated pairs
datafeat = data.drop(columns=['price_log','price'])
dtfc = datafeat.corr().abs().stack().reset_index().sort_values(0, ascending=False)

dtfc['col_pairs'] = list(zip(dtfc.level_0,dtfc.level_1))
dtfc['same'] = dtfc['col_pairs'].map(lambda x: (x[0] in x[1]) or (x[1] in x[0]))
dtfc['col_pairs'] = dtfc['col_pairs'].map(lambda x:sorted(list(x)))
dtfc.set_index(['col_pairs'],inplace=True)
dtfc = dtfc[dtfc['same'] == False]
dtfc.drop(columns=['level_0','level_1','same'],inplace=True)
dtfc.columns = ['C']
dtfc.drop_duplicates(inplace=True)
dtfc.head(20)

## Feature Engineering

Because these is some clear multicollinearity occurring between house features, we decided that one way to get around this was to add features that combined them in logical ways. This would help deal with the dependence of variables on each other, and maybe give some heightened insight into how these variables are important to the sale price.

In [ ]:
#bedrooms per sqfoot of living space
data["bedroom/sqft_above_log"] = data["bedrooms"] / data["sqft_above_log"]
#bathrooms per sqfoot of living space 
data["bathrooms/sqft_above_log"] = data["bathrooms"] / data["sqft_above_log"]

## Exploration of data correlations

In [ ]:
#checking the column names for our combined data sets
data.columns

## Exploring direct linear regressions on price
We will not explore logged data for clarity and understanding

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(data["sqft_living"],data['price'])

fig, ax = plt.subplots(figsize = (10,10))
ax = sns.regplot(x=(data["sqft_above"]), y=(data['price']/1000),
                 line_kws={'label':"${0:.2f}/Square foot increase in home space".format(slope)})
ax.legend()

In [ ]:
bathtemp = data[['price','bathrooms']]
bathtemp.loc[bathtemp['bathrooms'] <= 1,'bathrooms'] = 1
bathtemp.loc[(bathtemp['bathrooms'] > 1) & (bathtemp['bathrooms'] <= 2),'bathrooms'] = 2
bathtemp.loc[(bathtemp['bathrooms'] > 2) & (bathtemp['bathrooms'] <= 3),'bathrooms'] = 3
bathtemp.loc[(bathtemp['bathrooms'] > 3) & (bathtemp['bathrooms'] <= 4),'bathrooms'] = 4
bathtemp.loc[(bathtemp['bathrooms'] > 4) & (bathtemp['bathrooms'] <= 5),'bathrooms'] = 5
bathtemp.loc[(bathtemp['bathrooms'] > 5) & (bathtemp['bathrooms'] <= 6),'bathrooms'] = 6
bathtemp.loc[(bathtemp['bathrooms'] > 6) & (bathtemp['bathrooms'] <= 7),'bathrooms'] = 7
bathtemp.loc[(bathtemp['bathrooms'] > 7) & (bathtemp['bathrooms'] <= 8),'bathrooms'] = 8

In [ ]:
fig,ax = plt.subplots(figsize = (35,10))


sns.violinplot(x=bathtemp['bathrooms'],y=(bathtemp['price']/1000), data = bathtemp, 
               palette="rocket")

ax.set_title("Number of bathrooms effect on home price",fontsize=50)
ax.set_xlabel("Bathrooms",fontsize=40)
ax.set_ylabel("Price (in Thousands)",fontsize=40)
ax.tick_params(labelsize=30)
ax.yaxis.set_major_formatter('${x:1.0f}')
plt.savefig("bathrooms.png")

In [ ]:
fig,ax = plt.subplots(figsize = (25,10))
sns.violinplot(x='bedrooms',y=(data['price']/1000), data = data, palette="rocket")

ax.set_title("Number of bedrooms effect on home price",fontsize=50)
ax.set_xlabel("Bedrooms",fontsize=40)
ax.set_ylabel("Price (in Thousands)",fontsize=40)
ax.tick_params(labelsize=30)
ax.yaxis.set_major_formatter('${x:1.0f}')
plt.savefig("bedrooms.png")

## Let's build models.

### Train-Test Split the data
Here we split the data into a test and train set. We will fit and transform the training data and later fit the training data to analyze. We will be using he log transformed data to utilize the more normative distribution of the data. We will additionally be scaling our data to allow the model to weigh the features equally bcasue they will be on the same scale and we will be able to compare the feature coefficients bewteen one another.

In [ ]:
#X will b all of the features of the data set to analyze. "price_log" will be the target variable.
X = data.drop(columns=['price_log','price'])

X_train, X_test, y_train, y_test = \
train_test_split(X, data['price_log'], test_size=0.33, random_state=40)

In [ ]:
#looking at the descriptive statistics of our x training data.
X_train.describe()

In [ ]:
#looking at the descriptive statistics of our x test data.
X_test.describe()

In [ ]:
#looking at the descriptive statistics of our y training data.
y_train.describe()

In [ ]:
#looking at the descriptive statistics of our y test data.
y_test.describe()

In [ ]:
#Here we use SkLearn's Standard Scaler to scale all of our data to the same scale

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

In [ ]:
#taking a look at the scaled X trained data
X_train_scaled

## Baseline Understanding

The simplest model will predict the average price for all homes regardless of their features. This is the baseline model we use to compare all future models against.

In [ ]:
train_target_mean = y_train.mean()
baseline_train_pred = [train_target_mean] * len(y_train)
baseline_test_pred = [train_target_mean] * len(y_test)

In [ ]:
# function to evaluate the models and their predicted sale prices vs. the actual sale prices
def evaluate(y_tr, y_te, y_tr_pr, y_te_pr, log=True):
    '''
    Evaluates the error between the model predictions and the real values for both
    training and test sets.
    
    Arguments:
    y_tr - array-like
        Actual values for output variable, for the training set
    y_tr_pr - array-like
        Predicted values for output variable, for the training set
    y_te - array-like
        Actual values for output variable, for the test set
    y_te_pr - array-like
        Predicted values for output variable, for the test set
    log=True
        If true, 
    Returns:
    R2 scores for Train and Test sets
    RMSE for Train and Test sets
    MAE for Train and Test sets
    '''
    if log == True:
        y_tr = np.exp(y_tr)
        y_te = np.exp(y_te)
        y_tr_pr = np.exp(y_tr_pr)
        y_te_pr = np.exp(y_te_pr)
        
    # residuals
    train_res = y_tr - y_tr_pr
    test_res = y_te - y_te_pr
    
    print(f'Train R2 score: {r2_score(y_tr, y_tr_pr)} ')
    print(f'Test R2 score: {r2_score(y_te, y_te_pr)} ')
    print('<><><><><>')
    print(f'Train RMSE: ${mean_squared_error(y_tr, y_tr_pr, squared=False):,.2f} ')
    print(f'Test RMSE: ${mean_squared_error(y_te, y_te_pr, squared=False):,.2f} ')
    print('<><><><><>')
    print(f'Train MAE: ${mean_absolute_error(y_tr, y_tr_pr):,.2f} ')
    print(f'Test MAE: ${mean_absolute_error(y_te, y_te_pr):,.2f} ')
    

    
    # scatter plot of residuals
    print("\nScatter of residuals:")
    plt.scatter(y_tr_pr, train_res, label='Train')
    plt.scatter(y_te_pr, test_res, label='Test')
    plt.axhline(y=0, color='purple', label='0')
    plt.xlabel("Predicted Price")
    plt.ylabel("Residual Price")
    plt.legend()
    plt.show()
    
    print("QQ Plot of residuals:")
    fig, ax = plt.subplots()
    sm.qqplot(train_res, ax=ax, marker='.', color='r', label='Train', alpha=0.3, line='s')
    sm.qqplot(test_res, ax=ax,  marker='.', color='g', label='Test', alpha=0.3)
    plt.legend()

In [ ]:
evaluate(y_train, y_test, baseline_train_pred, baseline_test_pred)

This very simple model will predict the average of the training data set prices and will not utilize any independent variable. This will result in the residuals being the same. As you can see the training and test residuals are almost directly over laid on one another. Which is predicatble based on the paramaters of our current model. This is not a practical model for the scope of our project.

## Modeling

Describe and justify the process for analyzing or modeling the data.

Questions to consider:

- How did you analyze the data to arrive at an initial approach?
- How did you iterate on your initial approach to make it better?
- Why are these choices appropriate given the data and the business problem?

## Evaluation

### First model

In [ ]:
def smols(X,y,cols=None):
    '''
    Uses Linear Regression to find a best fit given desired features.
    Arguments:
    X - dataframe
        Input features and values
    y - array-like
        Output values
    cols=None - list
        List of features to be included from the X dataframe
    Returns: OLS model. Use smols().summary() to view summary
    '''
    Xcol = X[cols]
    shmod = sm.OLS(endog=y, exog=sm.add_constant(Xcol)).fit()
    return shmod

In [ ]:
def linpreds(X_tr_scaled, y_tr, X_te_scaled):
    '''
    Uses Linear Regression to generate output predictions given training and test inputs.
    Arguments:
    X_tr_scaled - dataframe
        Input variables and values for the training set
    y_tr - array-like
        Actual values for output variable, for the training set
    X_te_scaled - dataframe
        Input variables and values for the test set
    Returns:
    Output (y) prediction arrays:
        train, test
    '''
    lr = LinearRegression()
    lr.fit(X_tr_scaled, y_tr)
    return lr.predict(X_tr_scaled), lr.predict(X_te_scaled)

This very simple, substandard model is a simple linear regression model between the space in a home and the price of the home. This was the strongest correlation we found in our initial analysis. 

In [ ]:
cols = ['sqft_living_log']
smols(X_train,y_train,cols).summary()

This model is better than the baseline model, because it is taking one independent variable into the model compared to none in our base line model. The model is accounting for 43.2% variance in the model. The p-value is less than or alpha of 0.05 which implies significance. After scaling the data back the coefficient for the log square foot of the living space states acoounts for X amount of dollars of change for every 1 square foot increase in the home. 

### Modeling Iterations

Now that we have a 'better' model, we can start to add extra features that seem to add predictive value to the model. These are given by the correlation values generated earlier. We'll start with bathrooms and then add the grade categories. Sqft_above is ignored for now because it has some multicollinearity (independence) issues with sqft_living_log.

In [ ]:
smols(X_train_scaled,y_train,\
      cols=['sqft_living_log','bathrooms']).summary()

In [ ]:
smols(X_train_scaled,y_train,\
      cols=['sqft_living_log','bathrooms','Above Average Grade', 
            'Excellent Grade', 'Low Grade']).summary()

This model analyzes how home size, number of bathrooms, and grade interact to predict the home price. The r squred value has risen to 53.3% prediction in variance. Each of the pvalues is lower than alpha implying significance, except bathrooms, which is super high at 78.5%. This is surprising, given how strongly it alone is correlated with price. The top predictors are 'sqft_living_log' and 'Excellent Grade', which makes sense. Bigger homes in better quality should mean higher sale prices. Our condition number is low, meaning there are minimal issues with multicollinearity.

## Stepwise function

To find the best combination of features that result in the lowest error in predicted price, we built a forward-backward optimization function.

In the forward step, it takes a model with certain included features, and generates a Series of R2 values associated with models (using training data) that include the initial features and one extra feature among those remaining. The best R2 value (for test data) for the new model is compared to the R2 value (for test data) for the initial model. If it is higher, the new feature is added to the included features for the next step and next iteration.

In the backward step, after adding each new variable to the included-feature model, the algorithm will generate a Series of RMSE values associated with models that remove one variable from the included-feature model. In that list, the lowest RMSE is compared to the RMSE of the full included-feature model, and if it is lower, that feature is removed from the included-feature model for the next iteration. 

The algorithm proceeds in an iterative fashion until no features are added or removed.

In [ ]:
def stepwise_selection(X_tr, y_tr, X_te=None, y_te=None,
                       initial_list=[], no_use=[], 
                       verbose=True):
    """
    Perform a forward-backward feature selection 
    based on R2 (forward) and RMSE (backward) from sklearn
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        X_tr - pandas.DataFrame with training candidate features
        y_tr - list-like with the training target
        X_te - pandas.DataFrame with test candidate features
        y_te - list-like with the test target
        initial_list - list of features to start with (column names of X_tr)
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    """
    included = list(set(initial_list))
    while True:
        changed=False
        # forward step with R2
        # add feature if the resulting test R2 >= previous test R2
        on_hold = []
        excluded = list(set(X_tr.columns)-set(no_use)-set(included)-set(on_hold))
        new_r2 = pd.Series(index=excluded, dtype='float64')
        for new_column in excluded:
            trpred, tepred = linpreds(X_tr[included+[new_column]], y_tr, 
                                      X_te[included+[new_column]])
            new_r2[new_column] = r2_score(y_te, tepred)
        best_r2 = new_r2.max()
        if best_r2 > r2_score(y_te, tepred):
            best_feature = new_r2.idxmax()
            included.append(best_feature)
            try:
                on_hold.pop()
            except:
                pass
            changed=True
            if verbose:
                print('Add  {:30} with r2: {:.6}'.format(best_feature, best_r2))

        # backward step with RMSE
        trpred, tepred = linpreds(X_tr[included], y_tr, X_te[included])
        y_te_unl, tepred_unl = np.exp(y_te), np.exp(tepred)
        rmse_pre = mean_squared_error(y_te_unl, tepred_unl, squared=False)
        print('Before removal RMSE: {:.2f}'.format(rmse_pre))
        rmses = pd.Series(index=included, dtype='float64')
        for column in included:
            trpred, tepred = linpreds(X_tr[list(set(included)-set(column))], y_tr, 
                                      X_te[list(set(included)-set(column))])
            y_te_unl, tepred_unl = np.exp(y_te), np.exp(tepred)
            rmses[column] = mean_squared_error(y_te_unl, tepred_unl, squared=False)
        lowest_rmse = rmses.min()
        if lowest_rmse < rmse_pre:
            changed=True
            bad_feature = rmses.idxmin()
            on_hold.append(bad_feature)
            included.remove(bad_feature)
            if verbose:
                print('Drop {:30} with RMSE {:.2f}'.format(bad_feature, lowest_rmse))
        else:
            if verbose:
                print('Keep {:30} with RMSE {:.2f}'.format(best_feature, lowest_rmse))
        if not changed:
            break
    return included

In [ ]:
X_train.columns

In [ ]:
stepwise_selection(X_train_scaled, y_train, X_test_scaled, y_test,  \
                   initial_list=['sqft_living_log', 'Low Grade',
                                 'Excellent Grade','Above Average Grade'], 
                   no_use=['sqft_living','sqft_lot','sqft_above']) # don't include non-log features

### 'Final' Model

In the end, you'll arrive at a 'final' model - aka the one you'll use to make your recommendations/conclusions. This likely blends any group work. It might not be the one with the highest scores, but instead might be considered 'final' or 'best' for other reasons.

In [ ]:
relcol =['Excellent Grade',
 'Low Grade',
 'Above Average Grade',
 'sqft_living_log',
 'has_basement',
 'Good Condition',
 'sqft_lot_log']
smols(X_train_scaled, y_train, cols=relcol).summary()

In [ ]:
Xftr, Xfte = X_train_scaled[relcol], X_test_scaled[relcol]
trp, tep = linpreds(Xftr, y_train, Xfte)
evaluate(y_train, y_test, trp, tep)

Our final model resulted in a $152,7285.76 average variation from observed sales prices and ended with a R squared value of .555 meaning our model accounts for a 55.5% variance in sales price. All of the featurres have a pvalue of lesss than 0.05, which implies all features are significant to the model. Our condition number is less than a 5, meaning there is little to multicollinearity issues. The scatter plot of the residuals does display some heteroskedasticity. Based on the QQ plot, the residuals are slightly skewed to the right. It is interesting to note, none of the engineered features, the number of bathrooms did not make it in to the final model. 

## Conclusions

We recommend Mom & Pop Realty use the grade and condition of the house if the house has a basement, the size of the house and size of the property to predict the price of the a clients home. The strongest predictors is House Square Footage, where a 1% increase in Home quare footage translates to an increase in 0.22% sale price. The next strogest predictor is the grade of the house, specifically, where the house has an excellent grade. Homes with an excellent grade has a 20.1% higher sales price than that of a home with an average grade. We understand this model is incomplete and the level of bias in the model reduces the overall effectiveness. Our final model does not included location data. Adding those features to the model may help the model's bias and heteroskedasticity issues. 